# Hosting Strands Agents with Amazon Bedrock models in Amazon Bedrock AgentCore Runtime

## Overview

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. We will provide examples using Amazon Bedrock models and non-Bedrock models such as Azure OpenAI and Gemini.


### Tutorial Details


| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Single                                                                           |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Sonnet 4                                                        |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands Agent and Amazon Bedrock Model |
| Tutorial vertical   | Cross-vertical                                                                   |
| Example complexity  | Easy                                                                             |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                     |

### Tutorial Architecture

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime. 

For demonstration purposes, we will  use a Strands Agent using Amazon Bedrock models

In our example we will use a very simple agent with two tools: `get_weather` and `get_time`. 

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Amazon Bedrock models
* Using Strands Agents


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker running

In [ ]:
#!uv add -r requirements.txt --active

## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

The architecture here will look as following:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="50%"/>
</div>

In [1]:
%%writefile strands_claude_stream.py
import os
import json
import argparse
import asyncio
from datetime import datetime
from typing import Dict, Any

from strands import Agent
from strands.models import BedrockModel
from botocore.config import Config

# You'll need to import these from your project
# from src.utils.bedrock import bedrock_info

class bedrock_info:
    @staticmethod
    def get_model_id(model_name):
        # Placeholder - replace with actual implementation
        model_mapping = {
            "Claude-V3-5-V-2-Sonnet-CRI": "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
            "Claude-V3-7-Sonnet-CRI": "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        }
        return model_mapping.get(model_name, model_name)

def get_model(**kwargs):
    llm_type = kwargs["llm_type"]
    cache_type = kwargs["cache_type"]
    enable_reasoning = kwargs["enable_reasoning"]

    if llm_type == "reasoning":    
        llm = BedrockModel(
            model_id=bedrock_info.get_model_id(model_name="Claude-V3-7-Sonnet-CRI"),
            streaming=True,
            max_tokens=8192*5,
            stop_sequences=["\n\nHuman"],
            temperature=1 if enable_reasoning else 0.01, 
            additional_request_fields={
                "thinking": {
                    "type": "enabled" if enable_reasoning else "disabled", 
                    **({"budget_tokens": 8192} if enable_reasoning else {}),
                }
            },
            cache_prompt=cache_type,
            boto_client_config=Config(
                read_timeout=900,
                connect_timeout=900,
                retries=dict(max_attempts=50, mode="adaptive"),
            )
        )
        
    elif llm_type == "basic":
        llm = BedrockModel(
            model_id=bedrock_info.get_model_id(model_name="Claude-V3-5-V-2-Sonnet-CRI"),
            streaming=True,
            max_tokens=8192,
            stop_sequences=["\n\nHuman"],
            temperature=0.01,
            cache_prompt=cache_type,
            boto_client_config=Config(
                read_timeout=900,
                connect_timeout=900,
                retries=dict(max_attempts=50, mode="standard"),
            )
        )
    else:
        raise ValueError(f"Unknown LLM type: {llm_type}")
        
    return llm

class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

def apply_prompt_template(prompt_name: str, prompt_context={}) -> str:
    try:
        system_prompts = open(os.path.join("./prompts", f"{prompt_name}.md")).read()    
    except FileNotFoundError:
        # Fallback system prompt
        system_prompts = "You are a helpful AI assistant."
    
    context = {"CURRENT_TIME": datetime.now().strftime("%a %b %d %Y %H:%M:%S %z")}
    context.update(prompt_context)
    system_prompts = system_prompts.format(**context)
    return system_prompts

def get_agent(**kwargs):
    agent_name, system_prompts = kwargs["agent_name"], kwargs["system_prompts"]
    agent_type = kwargs.get("agent_type", "basic")
    prompt_cache_info = kwargs.get("prompt_cache_info", (False, None))
    tools = kwargs.get("tools", None)
    streaming = kwargs.get("streaming", True)
        
    if "reasoning" in agent_type: 
        enable_reasoning = True
    else: 
        enable_reasoning = False

    prompt_cache, cache_type = prompt_cache_info
    if prompt_cache: 
        print(f"{Colors.GREEN}{agent_name.upper()} - Prompt Cache Enabled{Colors.END}")
    else: 
        print(f"{Colors.GREEN}{agent_name.upper()} - Prompt Cache Disabled{Colors.END}")

    llm = get_model(llm_type=agent_type, cache_type=cache_type, enable_reasoning=enable_reasoning)
    llm.config["streaming"] = streaming

    agent = Agent(
        model=llm,
        system_prompt=system_prompts,
        tools=tools,
        callback_handler=None
    )
    return agent

async def _convert_to_agentcore_event(
    strands_event: Dict[str, Any],
    agent_name: str,
    session_id: str
) -> Dict[str, Any]:
    """Strands 이벤트를 AgentCore 스트리밍 형식으로 변환"""
    
    base_event = {
        "timestamp": datetime.now().isoformat(),
        "session_id": session_id,
        "agent_name": agent_name,
        "source": "strands_data_analysis_graph"
    }
    
    # 텍스트 데이터 이벤트
    if "data" in strands_event:
        return {
            **base_event,
            "type": "agent_text_stream",
            "event_type": "text_chunk",
            "data": strands_event["data"],
            "chunk_size": len(strands_event["data"])
        }
    
    # 도구 사용 이벤트
    elif "current_tool_use" in strands_event:
        tool_info = strands_event["current_tool_use"]
        return {
            **base_event,
            "type": "agent_tool_stream",
            "event_type": "tool_use",
            "tool_name": tool_info.get("name", "unknown"),
            "tool_id": tool_info.get("toolUseId"),
            "tool_input": tool_info.get("input", {})
        }
    
    # 추론 이벤트
    elif "reasoning" in strands_event and strands_event.get("reasoning"):
        return {
            **base_event,
            "type": "agent_reasoning_stream",
            "event_type": "reasoning",
            "reasoning_text": strands_event.get("reasoningText", "")[:200]
        }
    
    return None

async def process_agent_stream(agent, message):
    coordinator_result = ""
    agent_stream = agent.stream_async(message)
    session_id = "123"

    async for event in agent_stream:
        #Strands 이벤트를 AgentCore 형식으로 변환
        agentcore_event = await _convert_to_agentcore_event(
            event, "coordinator", session_id
        )
        if agentcore_event:
            yield agentcore_event

            # 결과 텍스트 누적
            if agentcore_event.get("event_type") == "text_chunk":
                coordinator_result += agentcore_event.get("data", "")

async def node(agent, message):
    async for event in process_agent_stream(agent, message):
        yield event

# Create agent instance
agent = get_agent(
    agent_name="task_agent",
    system_prompts=apply_prompt_template(prompt_name="task_agent", prompt_context={}),
    agent_type="reasoning",
    prompt_cache_info=(True, "default"),
    streaming=True,
)

async def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    async for event in node(agent, user_input):
        #print(f"Event: {event}")
        yield event

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    
    async def main():
        async for event in strands_agent_bedrock(json.loads(args.payload)):
            print(f"Final event: {event}")
    
    asyncio.run(main())

Overwriting strands_claude_stream.py


#### Invoking local agent

In [2]:
!python strands_claude_stream.py '{"prompt": "나는 장동진이야"}'

TASK_AGENT - Prompt Cache Enabled
Final event: {'timestamp': '2025-08-23T11:12:52.176343', 'session_id': '123', 'agent_name': 'coordinator', 'source': 'strands_data_analysis_graph', 'type': 'agent_reasoning_stream', 'event_type': 'reasoning', 'reasoning_text': 'The'}
Final event: {'timestamp': '2025-08-23T11:12:52.176569', 'session_id': '123', 'agent_name': 'coordinator', 'source': 'strands_data_analysis_graph', 'type': 'agent_reasoning_stream', 'event_type': 'reasoning', 'reasoning_text': ' user has'}
Final event: {'timestamp': '2025-08-23T11:12:52.176851', 'session_id': '123', 'agent_name': 'coordinator', 'source': 'strands_data_analysis_graph', 'type': 'agent_reasoning_stream', 'event_type': 'reasoning', 'reasoning_text': ' greeted me'}
Final event: {'timestamp': '2025-08-23T11:12:52.176884', 'session_id': '123', 'agent_name': 'coordinator', 'source': 'strands_data_analysis_graph', 'type': 'agent_reasoning_stream', 'event_type': 'reasoning', 'reasoning_text': ' in Korean'}
Final eve

## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### Strands Agents with Amazon Bedrock model
Let's start with our Strands Agent using Amazon Bedrock model. All the others will work exactly the same.

In [ ]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel

app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

In [17]:
%%writefile strands_claude_stream.py
import os
import json
import argparse
import asyncio
from datetime import datetime
from typing import Dict, Any

from strands import Agent
from strands.models import BedrockModel
from botocore.config import Config


from bedrock_agentcore.runtime import BedrockAgentCoreApp
app = BedrockAgentCoreApp()


# You'll need to import these from your project
# from src.utils.bedrock import bedrock_info

class bedrock_info:
    @staticmethod
    def get_model_id(model_name):
        # Placeholder - replace with actual implementation
        model_mapping = {
            "Claude-V3-5-V-2-Sonnet-CRI": "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
            "Claude-V3-7-Sonnet-CRI": "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        }
        return model_mapping.get(model_name, model_name)

def get_model(**kwargs):
    llm_type = kwargs["llm_type"]
    cache_type = kwargs["cache_type"]
    enable_reasoning = kwargs["enable_reasoning"]

    if llm_type == "reasoning":    
        llm = BedrockModel(
            model_id=bedrock_info.get_model_id(model_name="Claude-V3-7-Sonnet-CRI"),
            streaming=True,
            max_tokens=8192*5,
            stop_sequences=["\n\nHuman"],
            temperature=1 if enable_reasoning else 0.01, 
            additional_request_fields={
                "thinking": {
                    "type": "enabled" if enable_reasoning else "disabled", 
                    **({"budget_tokens": 8192} if enable_reasoning else {}),
                }
            },
            cache_prompt=cache_type,
            boto_client_config=Config(
                read_timeout=900,
                connect_timeout=900,
                retries=dict(max_attempts=50, mode="adaptive"),
            )
        )
        
    elif llm_type == "basic":
        llm = BedrockModel(
            model_id=bedrock_info.get_model_id(model_name="Claude-V3-5-V-2-Sonnet-CRI"),
            streaming=True,
            max_tokens=8192,
            stop_sequences=["\n\nHuman"],
            temperature=0.01,
            cache_prompt=cache_type,
            boto_client_config=Config(
                read_timeout=900,
                connect_timeout=900,
                retries=dict(max_attempts=50, mode="standard"),
            )
        )
    else:
        raise ValueError(f"Unknown LLM type: {llm_type}")
        
    return llm

class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

def apply_prompt_template(prompt_name: str, prompt_context={}) -> str:
    try:
        system_prompts = open(os.path.join("./prompts", f"{prompt_name}.md")).read()    
    except FileNotFoundError:
        # Fallback system prompt
        system_prompts = "You are a helpful AI assistant."
    
    context = {"CURRENT_TIME": datetime.now().strftime("%a %b %d %Y %H:%M:%S %z")}
    context.update(prompt_context)
    system_prompts = system_prompts.format(**context)
    return system_prompts

def get_agent(**kwargs):
    agent_name, system_prompts = kwargs["agent_name"], kwargs["system_prompts"]
    agent_type = kwargs.get("agent_type", "basic")
    prompt_cache_info = kwargs.get("prompt_cache_info", (False, None))
    tools = kwargs.get("tools", None)
    streaming = kwargs.get("streaming", True)
        
    if "reasoning" in agent_type: 
        enable_reasoning = True
    else: 
        enable_reasoning = False

    prompt_cache, cache_type = prompt_cache_info
    if prompt_cache: 
        print(f"{Colors.GREEN}{agent_name.upper()} - Prompt Cache Enabled{Colors.END}")
    else: 
        print(f"{Colors.GREEN}{agent_name.upper()} - Prompt Cache Disabled{Colors.END}")

    llm = get_model(llm_type=agent_type, cache_type=cache_type, enable_reasoning=enable_reasoning)
    llm.config["streaming"] = streaming

    agent = Agent(
        model=llm,
        system_prompt=system_prompts,
        tools=tools,
        callback_handler=None
    )
    return agent

async def _convert_to_agentcore_event(
    strands_event: Dict[str, Any],
    agent_name: str,
    session_id: str
) -> Dict[str, Any]:
    """Strands 이벤트를 AgentCore 스트리밍 형식으로 변환"""
    
    base_event = {
        "timestamp": datetime.now().isoformat(),
        "session_id": session_id,
        "agent_name": agent_name,
        "source": "strands_data_analysis_graph"
    }
    
    # 텍스트 데이터 이벤트
    if "data" in strands_event:
        return {
            **base_event,
            "type": "agent_text_stream",
            "event_type": "text_chunk",
            "data": strands_event["data"],
            "chunk_size": len(strands_event["data"])
        }
    
    # 도구 사용 이벤트
    elif "current_tool_use" in strands_event:
        tool_info = strands_event["current_tool_use"]
        return {
            **base_event,
            "type": "agent_tool_stream",
            "event_type": "tool_use",
            "tool_name": tool_info.get("name", "unknown"),
            "tool_id": tool_info.get("toolUseId"),
            "tool_input": tool_info.get("input", {})
        }
    
    # 추론 이벤트
    elif "reasoning" in strands_event and strands_event.get("reasoning"):
        return {
            **base_event,
            "type": "agent_reasoning_stream",
            "event_type": "reasoning",
            "reasoning_text": strands_event.get("reasoningText", "")[:200]
        }
    
    return None

async def process_agent_stream(agent, message):
    coordinator_result = ""
    agent_stream = agent.stream_async(message)
    session_id = "123"

    async for event in agent_stream:
        #Strands 이벤트를 AgentCore 형식으로 변환
        agentcore_event = await _convert_to_agentcore_event(
            event, "coordinator", session_id
        )
        if agentcore_event:
            yield agentcore_event

            # 결과 텍스트 누적
            if agentcore_event.get("event_type") == "text_chunk":
                coordinator_result += agentcore_event.get("data", "")

async def node(agent, message):
    async for event in process_agent_stream(agent, message):
        yield event

# Create agent instance
agent = get_agent(
    agent_name="task_agent",
    system_prompts=apply_prompt_template(prompt_name="task_agent", prompt_context={}),
    agent_type="reasoning",
    prompt_cache_info=(True, "default"),
    streaming=True,
)

@app.entrypoint
async def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    async for event in node(agent, user_input):
        #print(f"Event: {event}")
        #yield f"ss:{event}"
        if "data" in event:
            yield event["data"]

if __name__ == "__main__":

    app.run()
    # parser = argparse.ArgumentParser()
    # parser.add_argument("payload", type=str)
    # args = parser.parse_args()
    
    # async def main():
    #     async for event in strands_agent_bedrock(json.loads(args.payload)):
    #         print(f"Final event: {event}")
    
    # asyncio.run(main())

Overwriting strands_claude_stream.py


## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCore Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Creating runtime role

Before starting, let's create an IAM role for our AgentCore Runtime. We will do so using the utils function pre-developed for you.

In [18]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.join(utils_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import create_agentcore_role

agent_name="agentcore_strands"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

sys.path[0]: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials
Role already exists -- deleting and creating it again
policies: {'PolicyNames': ['AgentCorePolicy'], 'IsTruncated': False, 'ResponseMetadata': {'RequestId': '9f9b4e3d-7cbf-4a88-9526-6c2f77655df5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 23 Aug 2025 11:23:34 GMT', 'x-amzn-requestid': '9f9b4e3d-7cbf-4a88-9526-6c2f77655df5', 'content-type': 'text/xml', 'content-length': '380'}, 'RetryAttempts': 0}}
deleting agentcore-agentcore_strands-role
recreating agentcore-agentcore_strands-role
attaching role policy agentcore-agentcore_strands-role


### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [19]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude_stream.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Entrypoint parsed: file=/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/strands_claude_stream.py, bedrock_agentcore_name=strands_claude_stream
Configuring BedrockAgentCore agent: agentcore_strands
Generated Dockerfile: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/Dockerfile
Generated .dockerignore: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/.dockerignore
Keeping 'agentcore_strands' as default agent
Bedrock AgentCore configured: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/Dockerfile'), dockerignore_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/.dockerignore'), runtime='Docker', region='us-west-2', account_id='615299776985', execution_role='arn:aws:iam::615299776985:role/agentcore-agentcore_strands-role', ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [20]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'agentcore_strands' to account 615299776985 (us-west-2)
Starting CodeBuild ARM64 deployment for agent 'agentcore_strands' to account 615299776985 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: agentcore_strands
✅ ECR repository available: 615299776985.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-agentcore_strands
Using execution role from config: arn:aws:iam::615299776985:role/agentcore-agentcore_strands-role


✅ Reusing existing ECR repository: 615299776985.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-agentcore_strands


✅ Execution role validation passed: arn:aws:iam::615299776985:role/agentcore-agentcore_strands-role
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: agentcore_strands
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-5b70b9d456
Reusing existing CodeBuild execution role: arn:aws:iam::615299776985:role/AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-5b70b9d456
Using .dockerignore with 44 patterns
Uploaded source to S3: agentcore_strands/20250823-112336.zip
Updated CodeBuild project: bedrock-agentcore-agentcore_strands-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 5.0s
🔄 PROVISIONING started (total: 5s)
✅ PROVISIONING completed in 10.1s
🔄 PRE_BUILD started (total: 15s)
✅ PRE_BUILD completed in 10.1s
🔄 BUILD started (total: 25s)
✅ BUILD completed in 60.3s
🔄 POST_BUILD started (total: 86s)
✅ POST_BUILD completed in 10.1s
🔄 CO

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [21]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: agentcore_strands


'READY'

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [22]:
for event in agentcore_runtime.invoke({"prompt": "안녕 나는 장동진이야"}):
    print ("11", event)

Invoking BedrockAgentCore agent 'agentcore_strands' via cloud endpoint


안녕하

세요,

 장

동진님

...

 to meet you.

 I am an AI

 assistant. How can

 I help

 you today?)

In [23]:
invoke_response = agentcore_runtime.invoke({"prompt": "안녕 나는 장동진이야"})
#invoke_response

Invoking BedrockAgentCore agent 'agentcore_strands' via cloud endpoint


안녕하세

요 장

동진님

, 다

...

 can I help you

? Do you have

 any questions or topics

 you'd like to

 discuss?)

In [24]:
invoke_response

{}

### Processing invocation results

We can now process our invocation results to include it in an application

In [ ]:
from IPython.display import Markdown, display
import json
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is 2+2?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                logger.info(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_iam_role['Role']['RoleName']
)

# Congratulations!